In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [2]:
anss = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=3,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
)

Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.


In [3]:
len(anss)

TypeError: object of type 'ANSS_MultiCatalog' has no len()

In [ ]:

dl_train = anss.train.get_dataloader( batch_size=200, shuffle=False)
dl_val = anss.val.get_dataloader(batch_size=200, shuffle=False)
dl_test = anss.test.get_dataloader(batch_size=200, shuffle=False)

In [ ]:
model = eq.models.RecurrentTPP(
    num_extra_features=2, # the number of extra features
    tau_mean=(
        len(anss.train[0])/(anss.train[0].t_end-anss.train[0].t_start)
    ), # the mean of the inter-event time distribution
)

In [ ]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type    | Params
------------------------------------------
0 | hypernet_time | Linear  | 3.2 K 
1 | hypernet_mag  | Linear  | 33    
2 | rnn           | GRU     | 3.6 K 
3 | dropout  

Sanity Checking: 0it [00:00, ?it/s]

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:103: UserWarning: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
  rank_zero_warn(
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the nu

Training: 0it [00:00, ?it/s]

AttributeError: 'Batch' has no attribute 'mag_bounds'